# Trajectory Analysis, Visualization and Manipulation

## Loading trajectory data

In [ ]:
import MDAnalysis as mda

# Load data
simulation = mda.Universe('adk_equilibrium\\adk4AKE.psf',
                          'adk_equilibrium\\1ake_007-nowater-core-dt240ps.dcd', 
                          in_memory=True)

# Inspect the simulation
print(len(simulation.atoms), len(simulation.trajectory))

In [ ]:
# Atom selections
CA_group = simulation.select_atoms("name CA")
print(len(CA_group.atoms))

In [ ]:
# Atomgroups read directly from the frame being processed
for frame in simulation.trajectory[:10]:
    print(CA_group.atoms.positions[0])


In [ ]:
# We are now on the first frame
simulation.trajectory[0]
print("Frame %s" % simulation.trajectory.frame)
print(CA_group.atoms.positions[0])

# This moves the simulation to frame 4
simulation.trajectory[4]
print("Frame %s" % simulation.trajectory.frame)

print(CA_group.atoms.positions[0])

## Root Mean Square Displacements (RMSD)
<br/>
<div style="text-align: justify"> 
Root Mean Square Deviation (RMSD) evaluates the similarity between proteins or molecular structures over time. It quantifies the average displacement between corresponding atoms in two structures, often a simulated and a reference structure. First it aligns the structures to minimize the distances and then computes the RMSD.
</div>

In [ ]:
from IPython.display import Image
Image(filename='RMSD.png', width=500)

<br/>
<div style="text-align: justify"> 
RMSD gives an idea of broad conformational changes and is often used to observe the presence of structure rearrengements or displacements. It can also be used to have a first pick into the convergence of a simulation. RMSD profiles tend to increase at the beggining and then reach a plateau. 
</div>
<br/>
<div style="text-align: justify"> 
With Gromacs it can be computed using the gmx rms command:
</div>
<br/>
gmx rms -s file.tpr -f file.xtc -n file.ndx -o output.xvg    


In [ ]:
from MDAnalysis.analysis import rms, align
import pandas as pd

# Align trajectory to itself
aligner = align.AlignTraj(simulation, reference=simulation, select="backbone", in_memory=True)

# RMSD can be calculated from atom selection or from trajectories by giving selection atoms
rmsd_ca = rms.RMSD(CA_group, reference=CA_group, ref_frame=0).run()
rmsd_ca2 = rms.RMSD(simulation, reference=simulation, select="name CA", ref_frame=0).run()
# It's important to call the method .run() to obtain the results

# check that every point matches
False in (rmsd_ca.results.rmsd == rmsd_ca2.results.rmsd)

In [ ]:
# Load results into a dataframe (Table)
rmsd_df = pd.DataFrame(rmsd_ca.results.rmsd, 
                       columns=['Frame', 'Time','CA Backbone'])

rmsd_df

In [ ]:
# Add columns to the table
rmsd_df["Time (ns)"] = rmsd_df["Time"]/1000
rmsd_df

In [ ]:
import plotly.express as px

# Plotting
figure = px.line(rmsd_df, x="Time (ns)", y="CA Backbone")
figure.update_layout()

In [ ]:
# Compute estimates from the data
print(rmsd_df["CA Backbone"][1:].mean())
#print(rmsd_df["CA Backbone"][rmsd_df["Time (ns)"] > 20].mean())
equilibrium_frame = int(min(rmsd_df["Frame"][rmsd_df["Time (ns)"] > 20]))
print(equilibrium_frame)

## Root Mean Square Displacements (RMSD) pitfalls
<br/>
<div style="text-align: justify"> 
RMSD is usually not very informative. Simulations contain a high number of atoms which means that, by doing the square mean, relevant changes may get trapped into the noise. One solution could be to select subsets of atoms or to weight differently certain atoms.
</div>
<br/>
<div style="text-align: justify"> 
Another limitation is that RMSD suffers from degeneracy. Different conformation can have the same RMSD respect the reference frame. One can compute pairwise RMSD (expensive) or use other metrics.
</div>

<div style="text-align: justify"> 
For simulations that have not reached equilibrium special care should be used when analyzing RMSD profiles. Ideally one should discard all the non equilibirum frames.
</div>
</br>
<div style="text-align: justify"> 
RMSD differences that appear relatively early are usually due to the equilibration procedure. It is Important to use replicates and to careful inspect the simulations
</div>

## Root Mean Square Fluctuation (RMSF)
<br/>
<div style="text-align: justify"> 
Root Mean Square Fluctuation (RMSF) is a metric used to quantify the flexibility or mobility of atoms within a biomolecular system, such as a protein or a nucleic acid. It provides insight into how much individual atoms deviate from their average positions during the simulation.
</div>
<br/>
<div style="text-align: justify"> 
The first step is to align the simulation and compute an average structure. Then all the frames are aligned to the average positions and finally the RMSF can be computed.
</div>
<br/>
<div style="text-align: justify"> 
With Gromacs it can be computed with the gmx rmsf command.
</div>
<br/>
gmx rmsf -s file.tpr -f file.xtc -n file.ndx -o output.xvg    

In [ ]:
Image(filename='RMSF.png', width=500)

In [ ]:
# First aling and compute the average structure

average = align.AverageStructure(simulation, reference=simulation, select="name CA").run(start=equilibrium_frame)
ref = average.results.universe

aligner = align.AlignTraj(simulation, ref, select='name CA', in_memory=True).run()

In [ ]:
# Compute RMSF
rmsf_ca = rms.RMSF(CA_group).run(start=equilibrium_frame)

rmsf_df = pd.DataFrame(rmsf_ca.results.rmsf, 
                       columns=['Fluctuations'])

# Format data
rmsf_df

In [ ]:
rmsf_df["Atom number"] = CA_group.indices
rmsf_df

In [ ]:
# Plot
figure = px.line(rmsf_df, x="Atom number", y="Fluctuations")
figure.update_layout()

In [ ]:
# Visualization
# Add fluctuations on B-factor information
# Since gromacs files don't have B-factors we have to initialize the attirbute
simulation.add_TopologyAttr('tempfactors')

# Color all the atoms of each residue based on the fluctuations of its CA
protein = simulation.select_atoms('protein') # select protein atoms
for residue, r_value in zip(protein.residues, rmsf_ca.results.rmsf):
    residue.atoms.tempfactors = r_value

# Save pdb with b-factors as colors
simulation.atoms.write('rmsf_tempfactors.pdb')


In [ ]:
# Nglview to have a short pic of the results
import nglview as nv

view = nv.show_mdanalysis(simulation.atoms)
view.update_representation(color_scheme='bfactor')
view

## Root Mean Square Fluctuation (RMSF) pitfalls
<br/>
<div style="text-align: justify"> 
RMSF can require long timescales to properly converge, specially on highly mobile parts whose energy landscape contains deep minimas.</div>
<br/>
<div style="text-align: justify"> 
Since RMSF provides a single measure for the whole trajectory, parts of the protein that behave in a "multimodal way" (conformation with low fluctuations and a conformation with high fluctuations) may not be described properly. 
</div>

## Radius of gyration (R<u>g</u>)
<br/>
<div style="text-align: justify"> 
The radius of gyration (R<u>g</u>) of a structure provides a metric of its compactness. It is a useful metric to detect possible folding and unfolding events.</div>
<br/>
<div style="text-align: justify"> 
To compute the radius of gyration first it is necessary to compute the center of mass of the selected atoms and the distance of all the atoms from the center of mass (r<u>i</u>). Then the R<u>g</u> can be computed as follows: 
</div>
<br/>
<div style="text-align: justify"> 
With Gromacs it can be computed with the gmx gyrate command
</div>
<br/>
gmx gyrate -s file.tpr -f file.xtc -n file.ndx -o output.xvg    

In [ ]:
Image(filename='gyration.png', width=500)

In [ ]:
# Data holder for the radius of gyration and frames
gyration = []
time = []

for i, ts in enumerate(simulation.trajectory[equilibrium_frame:]):
    time.append(simulation.trajectory.time/1000)
    gyration.append(simulation.atoms.radius_of_gyration())

gyration_df = pd.DataFrame({"Time (ns)": time, "Gyration": gyration})
gyration_df

In [ ]:
# Plot
figure = px.line(gyration_df, x="Time (ns)", y="Gyration")
figure.update_layout()

## Principal component analysis or dimensionality reduction
<br/>
<div style="text-align: justify"> 
Principal Component Analysis (PCA) is a dimensionality reduction technique commonly used in various fields, including molecular dynamics (MD) simulations, to simplify complex data while retaining its most important features. In the context of MD simulations, PCA is used to analyze and visualize the high-dimensional trajectory data of a molecular system.
</div>
<br/>
<div style="text-align: justify"> 
PCA begins by calculating a covariance matrix from the centered data, revealing how different atom positions vary together.  Then an Eigenvalue decomposition of the covariance matrix is performed which yields eigenvectors (representing variance directions) and eigenvalues (indicating variance amounts). Then the principal components can be determined and sorted based on the ammount of varaince they explain. These define a new coordinate system capturing significant variability in the original data.
By selecting a subset of principal components explaining most variance (usually 2 or 3), data dimensionality is reduced while preserving critical information.
Finally, the original data is projected creating transformed values. This simplifies visualization and represents the MD trajectory in reduced dimensions.</div>
<br/> 

In [ ]:
from MDAnalysis.analysis import pca

# Align the trajectory
aligner = align.AlignTraj(simulation, simulation, select='name CA',
                          in_memory=True).run(start=equilibrium_frame)

# Compute PCA
pc = pca.PCA(simulation, select='name CA',
             align=True, mean=None,
             n_components=None).run(start=equilibrium_frame)


In [ ]:
# Inspect variance explained by the first component
print(pc.results.variance[0])

# Accumulative variance explained by the first 3 components
pc.results.cumulated_variance[3]

explained_variance_df = pd.DataFrame({"Eigenvector": [i+1 for i in range(10)],
                                      "Explained Variance": pc.results.cumulated_variance[:10]})

# plot explained variance of 10 first components
figure = px.line(explained_variance_df, x="Eigenvector", y="Explained Variance")
figure.update_layout()

In [ ]:
# Transform selection to project them to the selected components
transformed = pc.transform(CA_group, n_components=3)
# Create Dataframe
pca_df = pd.DataFrame(transformed[equilibrium_frame:],
                  columns=['PC{}'.format(i+1) for i in range(3)])
# add time colum
pca_df["Time (ns)"] = (pca_df.index + equilibrium_frame) * simulation.trajectory.dt/1000 

pca_df

In [ ]:
# 3d PCA plot
fig = px.scatter_3d(pca_df, x="PC1", y="PC2", z="PC3", color="Time (ns)")
fig.update_traces(**{"marker.size": 3})
fig

## Principal component analysis or dimensionality reduction
<br/>
<div style="text-align: justify"> 
Although PCA are really good exploratory analysis one should treat them with care. Due to the sampling timescales limitations that current computational resources have; it is practically impossible to have fully converged simulations. This means that a PCA analysis will be able to capture those differences, even if in reality they are not meaningful. When using PCA analysis is crucial to use replicates and to carefully asses convergence.
</div>
<br/>
<div style="text-align: justify"> 
Another common mistake occurs during the analysis itself. It is important, in order to compare PCA's, that the simulations have been projected on the same space. This can be achieved by concatenating the simulations first, running the analysis and then colour them based on the simulation that they provide from. </div>
<br/>
<div style="text-align: justify"> 
Because of this, it is of crucial importance that the atoms of the simulations being concatenated match and that there is no difference in the initial setup and orientation between conditions. Ideally the different conditions should have started from simulations that are as similar as possible to avoid the PCA from picking differences originating from the setup. </div>

In [ ]:
from MDAnalysis.core.universe import Merge
from MDAnalysis.coordinates.memory import MemoryReader
import numpy as np

# Example of how to create a concatenated simulation of points of interest

# Create a universe object
combined_simulation = Merge(simulation.select_atoms("name CA"))

# Extract coordinates from the simulations
coordinates1 = np.array([CA_group.positions.copy() for frame in simulation.trajectory[equilibrium_frame:]])
coordinates2 = np.array([CA_group.positions.copy() for frame in simulation.trajectory[equilibrium_frame:]])

# Load data into the combined simulation
combined_simulation.load_new(np.vstack([coordinates1, coordinates2]), format=MemoryReader)

## Other analysis techniques
<br/>
<div style="text-align: justify"> 
The techniques covered here encompass some of the standard analysis that one commonly uses when analysing MD simulations. However, there are plenty of other analysis available. Which analysis technique is more descriptive greatly depends on the particular study case. Finding a measurement that correctly describes and discriminates the process of interest can be challenging. 
</div>
<br/>
<div style="text-align: justify"> 
The experimental setup used can also greatly influence the results one may obtain. In addition to that, it is also possible to design more complex MD simulations that open the door to more complex analysis, such as enhanced sampling techniques and free energy calculation.
</div>